## Double Pendulum

To simplify the problem, I have set $L_1 = L_2 =L$ and $m_1 = m_2 = m$ for this topic.

the Lagrangian for this system is:

$\begin{align}
  \mathcal{L} = m L^2 \dot\phi_1^2 + m L^2 \dot\phi_1\dot\phi_2 \cos(\phi_1-\phi_2)+ \frac12 m L^2 \dot\phi_2^2 - 2m g L (1 - \cos\phi_1)-m g L (1 - \cos\phi_2)
\end{align}$

we have $\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_1} = \frac{\partial\mathcal L}{\partial\phi_1}\end{align}$, $\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_2} = \frac{\partial\mathcal L}{\partial\phi_2}\end{align}$,
 
 That is,
 
$\begin{align}
2m L^2 \ddot\phi_1 + m L^2 \ddot\phi_2 \cos(\phi_2 - \phi_1) - m L^2 \dot\phi_2^2 \sin(\phi_2-\phi_1) + 2m g L \sin\phi_1 = 0
  \;
\end{align}$

$\begin{align}
m L^2 \ddot\phi_2 + m L^2 \ddot\phi_1 \cos(\phi_2 - \phi_1) + m L^2 \dot\phi_1^2 \sin(\phi_2 - \phi_1) + m g L \sin\phi_2 = 0
  \;
\end{align}$

### Other Information

*I have set up a vector in the code that contains all the quantities we need*

*In this case I used state space to check if chaos is generated*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.integrate import odeint, solve_ivp
from matplotlib import animation
from IPython.display import HTML

In [ ]:
plt.rcParams['figure.dpi'] = 150.    # dpi means resolution, in general the higher the dpi the clearer the picture
font_size = 10
plt.rcParams.update({'font.size': font_size})

In [ ]:
class LagrangianDoublePendulum():
    """
    This class simulates the treatment of a double pendulum with Lagrange's equation.
    
    Note that there is no driving or damping of the double pendulum simulated in this class.
     
    Parameters
    ----------
    
    L : 
        Type: float
        Physical meaning: length of the each simple pendulum
        
    mass : 
        Type: float
        Physical meaning: mass of each pendulum
        
    g : 
        Type: float
        Physical meaning: gravitational acceleration at the earth's surface
        
    omega_0 : 
        Type: float
        Physical meaning: natural frequency of the pendulum (sqrt{g/l}, l is the 
        pendulum length) 

    Methods
    -------
    
    dy_dt(t, y)
        (note thet y = {y[0], y[1], y[2], y[3]}) 
        Returns right of the differential equation in vector y
        (ie.(dphi/dt, d^2phi/dt^2))
        
    """
    def __init__(self, L=1., m=1., g=1.
                ):
        self.L = L
        self.m = m
        self.g = g

    
    def dy_dt(self, t, y):
        """
        This function returns the right side of the differential equations
        (ie.(dphi/dt, d^2phi/dt^2) 
        
        Parameters
        ----------
        t : 
            Type: float
            Physical meaning: time 
        y : 
            Type:float
            A vector y, and y = {y[0], y[1], y[2], y[3]} 
                with y[0] = phi_1(t) and y[1] = dphi_1/dt
                     y[2] = phi_2(t) and y[3] = dphi_2/dt
            
        Returns
        -------
        
        """      
        dy = np.zeros(4) #Create an empty vector group with four components
        
        dy[0] = y[1]
        dy[1] = (self.m * self.g * np.sin(y[2]) * np.cos(y[0]-y[2]) - self.m * np.sin(y[0] - y[2]) * (self.L * y[1]**2 * np.cos(y[0] - y[2]) + self.L * y[3]**2) -
             (2 * self.m) * self.g * np.sin(y[0])) / self.L / (self.m + self.m * np.sin(y[0] - y[2])**2)
        dy[2] = y[3]
        dy[3] = ((2 * self.m)*(self.L * y[1]**2 * np.sin(y[0] - y[2]) - self.g * np.sin(y[2]) + self.g * np.sin(y[0]) * np.cos(y[0] - y[2])) + 
             self.m * self.L * y[3]**2 * np.sin(y[0] - y[2]) * np.cos(y[0] - y[2])) / self.L / (self.m + self.m * np.sin(y[0] - y[2])**2)
        
        return dy
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi_1, phi1_dot, phi_2, phi2_dot = solution.y

        return phi_1, phi1_dot, phi_2, phi2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    
    plotting function: Output a chart with Cartesian coordinate system 
    (i.e. x-axis and y-axis)
    
    Customizable title, axis labels, and line label, line color, chart style.
    
    """
    if ax is None:       
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    
        ax.legend()
    if title is not None:    
        ax.set_title(title)
    if axis_labels is not None:  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
# start and stop points of indices
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts - plot_start)).argmin() #start
    stop_index = (np.fabs(t_pts - plot_stop)).argmin()  #end
    return start_index, stop_index

### Now lets plots

In [ ]:
# set plotting time
t_start = 0.
t_end = 60.
delta_t = 0.01

# set time array
t_pts = np.arange(t_start, t_end + delta_t, delta_t)  

#Assigning a value to letters
L = 1.
g = 1.
m = 1.

# Instantiate a pendulum 
p1 = LagrangianDoublePendulum(L=L, m=m, g=g)

In [ ]:
#Creat labels for axes
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
state_space_labels = (r'$\phi$', r'$d\phi/dt$')
KE_vs_time_labels = (r'$t$', r'$total KE$')

# both plots: same initial conditions
phi1_0 = np.pi / 3
phi1_dot_0 = 0.
phi2_0 = np.pi / 2
phi2_dot_0 = 0.

phi1, phi1_dot, phi2, phi2_dot, = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)

# start the plot
fig = plt.figure(figsize=(15,10))
overall_title = 'Simple couple pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
#Here are the images we need
ax_a = fig.add_subplot(2,2,1)                  
start, stop = start_stop_indices(t_pts, t_start, t_end)    

plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=r'$\phi(t)$ vs time', 
            ax=ax_a)   
plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=r'$\phi(t)$ vs time', 
            ax=ax_a)  

#To facilitate checking the correctness of the system, 
#I draw out the KE-T diagram here
ax_a = fig.add_subplot(2,2,2)
start, stop = start_stop_indices(t_pts, t_start, t_end)    

plot_y_vs_x(t_pts[start : stop], (1 / 2) * ( phi2_dot[start : stop]**2 + phi1_dot[start : stop]**2), 
            axis_labels=KE_vs_time_labels, 
            color='red',
            label=None, 
            title=r'$Total KE$ vs time', 
            ax=ax_a)  

#Note that the initial condition we use is already beyond the small angle approximation
#Here the space state graph is used directly to detect whether chaos is generated
ax_a = fig.add_subplot(2,2,3)                  
start, stop = start_stop_indices(t_pts, t_start, t_end)    

plot_y_vs_x(phi1[start : stop], phi1_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space of phi1', 
            ax=ax_a)  

ax_a = fig.add_subplot(2,2,4) 
start, stop = start_stop_indices(t_pts, t_start, t_end)    

plot_y_vs_x(phi2[start : stop], phi2_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space of phi2', 
            ax=ax_a)  
                              

From the state space, we can find that chaos has been created

### Now let's create the animation

In [ ]:
class LagrangianDoublePendulum3():

    def __init__(self, L=1., m=1., g=1.
                ):
        self.L = L
        self.m = m
        self.g = g

    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi/dt d^2phi/dt^2]
        
        Parameters
        t : time 
        y : A 4-component vector with y[0] = phi_1(t) and y[1] = dphi_1/dt
                                      y[2] = phi_2(t) and y[3] = dphi_2/dt
                                      y[4] = phi_3(t) and y[5] = dphi_3/dt
        """      
        dy = np.zeros(6)
        dy[0] = y[1]
        dy[1] = (10 * self.g * np.sin(y[0]) + 4 * self.g * np.sin(y[0] - 2 * y[2]) - self.g * np.sin(y[0] + 2 * y[2] - 2 * y[4]) - self.g * np.sin(y[0] - 2 * y[2] + 2 * y[4]) + 4 * self.L * np.sin(2 * (y[0] - y[2])) * y[1]**2 + 8 * self.L * np.sin(y[0] -y[2]) * y[3]**2 + 2 * self.L * np.sin(y[0] - y[4]) * y[5]**2 + 2 * self.L * np.sin(y[0] - 2 * y[2] + y[4]) * y[5]**2) / (2 * self.L * (-5 + 2 * np.cos(2 * (y[0] - y[2])) + np.cos(2 * (y[2] - y[4]))  ))
        dy[2] = y[3]
        dy[3] = (-7 * self.g * np.sin(2 * y[0] - y[2]) + 7 * self.g * np.sin(y[2]) + self.g * np.sin(y[2] - 2 * y[4]) + self.g * np.sin(2 * y[0] + y[2] - 2 * y[4]) + 2 * self.L * (-7 * np.sin(y[0] - y[2])  + np.sin(y[0] + y[2] - 2 * y[4])) * y[1]**2 + 2 * self.L * (-2 * np.sin(2 * (y[0] - y[2])) + np.sin(2 * (y[2] - y[4]))) * y[3]**2 - 2 * self.L * np.sin(2 * y[0] - y[2] - y[4]) * y[5]**2 + 6 * self.L * np.sin(y[2] - y[4]) * y[5]**2)  / (2 * self.L * (-5 + 2 * np.cos(2 * (y[0] - y[2])) + np.cos(2 * (y[2] - y[4]))  ))
        dy[4] = y[5]
        dy[5] = - ((2 * np.sin(y[2] - y[4]) * (self.g * np.cos(2 * y[0] - y[2]) + self.g * np.cos(y[2]) + 2 * self.L * np.cos(y[0] - y[2]) * y[1]**2 + 2 * self.L * y[3]**2 + self.L * np.cos(y[2] -y[4]) * y[5]**2 )) / (self.L * (-5 + 2 * np.cos(2 * (y[0] - y[2])) + np.cos(2 * (y[2] - y[4])))))
      
        return dy
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0,
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi_1, phi1_dot, phi_2, phi2_dot, phi_3, phi3_dot   = solution.y

        return phi_1, phi1_dot, phi_2, phi2_dot, phi_3, phi3_dot

In [ ]:
# set plotting time
t_start = 0.
t_end = 60.
delta_t = 0.01

# set time array
t_pts = np.arange(t_start, t_end + delta_t, delta_t)  

#Assigning a value to letters
L = 1.
g = 1.
m = 1.

# Instantiate a pendulum 
p2 = LagrangianDoublePendulum3(L=L, m=m, g=g)

In [ ]:
phi1_0 = np.pi/4
phi1_dot_0 = 0.

phi2_0 = np.pi/3
phi2_dot_0 = 0.

phi3_0 = np.pi/2
phi3_dot_0 = 0.

phi1, phi1_dot, phi2, phi2_dot,phi3, phi3_dot = p2.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0)

In [ ]:
x1 = L*np.sin(phi1)      
y1 = -L*np.cos(phi1)
x2 = L*np.sin(phi2)+x1
y2 = y1-L*np.cos(phi2)
x3 = L*np.sin(phi3)+x2
y3 = y2 - L*np.cos(phi3)
# these are x,y coordinates of pendulum

In [ ]:
%%capture

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-3, 3), ylim=(-3, 3))
ax.set_aspect('equal')
ax.grid()

line, = ax.plot([], [], 'o-', lw=2)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def anim(i):
    iskip = 6*i
    
    thisx = [0,x1[iskip], x2[iskip],x3[iskip]]
    thisy = [0,y1[iskip], y2[iskip],y3[iskip]]
    
    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (iskip*delta_t))
    return line, time_text

In [ ]:
# anim = animation.FuncAnimation(fig, anim, np.arange(1, len(t_pts)),
#     interval=25, blit=True, init_func=init)

frame_interval = 1.  # time between frames
frame_number = 1001   # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig, 
                               anim, 
                               init_func=init,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
HTML(anim.to_jshtml())  # animate using javascript